In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
### Distribution ✨ ❇ Demo 💪 ### 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

### fancy logging variables, philosophically reminding us of the goal ###
fancy = dict(
		pe		= r'$V(X)',    				
		ke		= r'$\nabla^2',    		
		e		= r'$E',						
		log_psi	= r'$\log\psi', 			
		deltar	= r'$\delta_\mathrm{r}',	
		x		= r'$r_\mathrm{e}',
)

### pyfig ###
from pyfig import Pyfig

# arg = {
# 	'a_z':[4,], 
# 	'n_b': 256, 
# 	'n_sv': 16, 
# 	'n_pv': 16, 
# 	'n_corr': 20, 
# 	'n_step': 100000, 
# 	'log_metric_step': 1, 
# 	'exp_name':'demo',
# }

c = Pyfig(wb_mode='online', submit=False, run_sweep=False, notebook=True)

n_device = c.n_device
print(f'🤖 {n_device} GPUs available')


# from pprint import pprint
# pprint(c.d)

""" live plotting in another notebook """
""" copy lines and run in analysis while the exp is live """
# api = wandb.Api()
# run = api.run("<run-here>")
# c = run.config
# h = run.history()
# s = run.summary


# SOLVE THE CONUNDRUM 

init sub classes
updating configuration
Run: ['git', 'log', '--pretty=format:%h', '-n', '1'] at /home/amawi/projects/hwat
stdout: 193e83a stderr: 
Run: ['hostname'] at .
stdout: oceanus  stderr: 
running script
setting exp_path
Run: ['git', 'log', '--pretty=format:%h', '-n', '1'] at /home/amawi/projects/hwat
stdout: 193e83a stderr: 
Run: ['hostname'] at .
stdout: oceanus  stderr: 


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xmax1 (hwat). Use `wandb login --relogin` to force relogin


🤖 1 GPUs available


' copy lines and run in analysis while the exp is live '

In [3]:
import torch
torch.manual_seed(1234)
torch.set_default_tensor_type(torch.DoubleTensor)   # ❗ Ensure works when default not set AND can go float32 or 64
cuda = torch.cuda.is_available()
n_device = torch.cuda.device_count()
current_device = torch.cuda.current_device()
device = torch.cuda.device(0)
device_name = torch.cuda.get_device_name(0)
print('cuda: ', cuda, 'n_dev: ', n_device, 'device', device, 'name: ', device_name)

cuda:  True n_dev:  1 device <torch.cuda.device object at 0x7f58fc417ee0> name:  NVIDIA TITAN Xp


In [4]:
device = 'cuda'

In [6]:
### model ###
from functools import partial
from hwat_func import Ansatz_fb
from torch import nn

import pprint

from hwat_func import init_r, get_center_points
x = torch.randn(1)
c._convert(device, dtype=x.dtype)
n_e = c.data.n_e
center_points = get_center_points(c.data.n_e, c.data.a)
r = init_r(n_device, c.data.n_b, c.data.n_e, center_points, std=0.1).to(device)
dtype = r.dtype

print('r: ', r.shape, r.dtype)
r = r[0] # single batch, single gpu, ❗ how to multi gpu
model = c.partial(Ansatz_fb).to(device)

model_check = c.partial(Ansatz_fb, with_sign=True).to(device)
r_swap = r[0, [1,0]+[i for i in range(2, c.data.n_e)]]
lp0, s0 = model_check(r[0])
lp1, s1 = model_check(r_swap)
print('anti-symmetry: ', lp0.item(), lp0.item(), s0.item(), s1.item())
r_swap = r[0, [i for i in range(0, c.data.n_e-2)]+[n_e-1,n_e-2]]
lp1, s1 = model_check(r_swap)
print('anti-symmetry: ', lp0.item(), lp0.item(), s0.item(), s1.item())

pprint.pprint(c.d)

r:  torch.Size([1, 256, 4, 3]) torch.float64
Run: ['git', 'log', '--pretty=format:%h', '-n', '1'] at /home/amawi/projects/hwat
stdout: 193e83a stderr: 
Run: ['hostname'] at .
stdout: oceanus  stderr: 
Run: ['git', 'log', '--pretty=format:%h', '-n', '1'] at /home/amawi/projects/hwat
stdout: 193e83a stderr: 
Run: ['hostname'] at .
stdout: oceanus  stderr: 
anti-symmetry:  -14.735963386332884 -14.735963386332884 -1.0 1.0
anti-symmetry:  -14.735963386332884 -14.735963386332884 -1.0 1.0
Run: ['git', 'log', '--pretty=format:%h', '-n', '1'] at /home/amawi/projects/hwat
stdout: 193e83a stderr: 
Run: ['hostname'] at .
stdout: oceanus  stderr: 
{'TMP': PosixPath('dump/tmp'),
 'cap': 3,
 'commit_id': '193e83a',
 'data': {'a': tensor([[0., 0., 0.]], device='cuda:0'),
          'a_z': tensor([4.], device='cuda:0'),
          'acc_target': 0.5,
          'charge': 0,
          'n_b': 256,
          'n_corr': 20,
          'n_d': 2,
          'n_e': 4,
          'n_equil': 10000,
          'n_u': 2,


In [13]:
### train step ###
from hwat_func import compute_ke_b, compute_pe_b
from functorch import vmap, make_functional, grad

def train_step(model, r):

	print(r.shape, model.a.shape, model.a)
	model_fn, params = make_functional(model)
	model_v = vmap(model_fn, in_dims=(None, 0))

	model_gv = lambda _params: model_v(_params, r)
	model_rv = lambda _r: model_v(params, _r)
 
	# with torch.no_grad():
	ke = compute_ke_b(model_rv, r)
	pe = compute_pe_b(r, c.data.a, c.data.a_z)
	# print(pe.shape, ke.shape)
	e = -0.5*ke + pe 
	e_mean_dist = torch.mean(torch.abs(torch.median(e) - e))
	# print(e.shape, e_mean_dist.shape)
	e_clip = torch.clip(e, min=e-5*e_mean_dist, max=e+5*e_mean_dist)

	loss = lambda _params: ((e_clip - e_clip.mean())*model_gv(_params)).mean()
	grad_fn = grad(loss)	
	grads = grad_fn(params)

	for p, g in zip(model.parameters(), grads):    # ❗ do we want functional or other design?! 
		p.grad = g.clone()
  
	# loss.backward()
	grads = [p.detach().cpu().numpy() for p in model.parameters()]
	params = [p.detach().cpu().numpy() for p in model.parameters()]  

	v_tr = dict(
		params=params, 
  		grads=grads,
		e=e, pe=pe, ke=ke,
		r=r
	)
	return v_tr

r = r.to(device)
center_points = center_points.to(device)

v_tr = train_step(model, r)
pprint.pprint({k:v.shape if isinstance(v, torch.Tensor) else len(v) for k,v in v_tr.items()})

### init variables ###
deltar = torch.tensor([0.02]).to(device)

print(f"""exp/actual | 
	cps    : {(c.data.n_e,3)}/{center_points.shape}
	r      : {(c.n_device, c.data.n_b, c.data.n_e, 3)}/{r.shape}
	deltar : {(1,)}/{deltar.shape}
""")

### init functions ### 
from hwat_func import sample_b

### train ###
import wandb
from hwat_func import keep_around_points
from utils import compute_metrix
from time import time
t0 = time()

### add in optimiser
opt = torch.optim.Adam(model.parameters(), lr=0.001)
c.log_metric_step = 1
### fix sampler
### fix train step 
### metrix conversion
# print(c.wandb_c.)
wandb.define_metric("*", step_metric="tr/step")
print('Go see:', c._run.url)
for step in range(1, c.n_step+1):
	
	model_fn, params = make_functional(model)
	model_v = vmap(model_fn, in_dims=(None, 0))
	model_rv = lambda _r: model_v(params, _r)
 
	r, acc, deltar = sample_b(model_rv, r, deltar, n_corr=c.data.n_corr)  # ❗needs testing  ✅
	r = keep_around_points(r, center_points, l=2.) if step < 1000 else r
	opt.zero_grad()
	v_tr = train_step(model, r)
	opt.step()
	
	if not (step % c.log_metric_step):
		metrix = compute_metrix(v_tr)  # ❗ needs converting to torch, ie tree maps ✅
		wandb.log({'tr/step':step, **metrix})
	
	diff = time()-t0
	if diff > 100: 
		t0 = time()
		pprint.pprint({k:f'{v.mean().item():.4f}' for k,v in v_tr.items() if isinstance(v, torch.Tensor)} | {'step': step})
  

torch.Size([256, 4, 3]) torch.Size([1, 3]) tensor([[0., 0., 0.]], device='cuda:0')
torch.Size([256, 4, 3]) jvp
torch.Size([4, 3]) torch.Size([1, 3])
torch.Size([4, 3]) torch.Size([1, 3])
{'e': torch.Size([256]),
 'grads': 20,
 'ke': torch.Size([256]),
 'params': 20,
 'pe': torch.Size([256]),
 'r': torch.Size([256, 4, 3])}
exp/actual | 
	cps    : (4, 3)/torch.Size([4, 3])
	r      : (1, 256, 4, 3)/torch.Size([256, 4, 3])
	deltar : (1,)/torch.Size([1])

Go see: https://wandb.ai/hwat/hwat/runs/1dnxy161


ValueError: vmap(FunctionalModule(
  (stateless_model): Ansatz_fb(
    (Vs): ModuleList(
      (0): Linear(in_features=20, out_features=32, bias=True)
      (1): Linear(in_features=128, out_features=32, bias=True)
      (2): Linear(in_features=128, out_features=32, bias=True)
    )
    (Ws): ModuleList(
      (0): Linear(in_features=4, out_features=16, bias=True)
      (1): Linear(in_features=16, out_features=16, bias=True)
      (2): Linear(in_features=16, out_features=16, bias=True)
    )
    (V_half_u): Linear(in_features=32, out_features=16, bias=True)
    (V_half_d): Linear(in_features=32, out_features=16, bias=True)
    (wu): Linear(in_features=16, out_features=2, bias=True)
    (wd): Linear(in_features=16, out_features=2, bias=True)
  )
), in_dims=(None, 0), ...)(<inputs>): in_dims is not compatible with the structure of `inputs`. in_dims has structure TreeSpec(tuple, None, [*, *]) but inputs has structure TreeSpec(tuple, None, [*]).

In [ ]:
# ```{toggle} env vars and jax debug config notes
# ❇️ Magic & debug not currently used

# %load_ext autoreload
# %autoreload 2
# %env CUDA_VISIBLE_DEVICES='3'
# %env "WANDB_NOTEBOOK_NAME" "run.ipynb" # ❕same as notebook

# from jax.config import config
# config.update('jax_disable_jit', True)
# ```